In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import re 
import difflib 



In [2]:
def cleaning(MVA_DF):
    print("original size : ", len(MVA_DF))
    MVA_DF= MVA_DF.reset_index(drop=True)
    reg = re.compile("((.*)?I(-|\s)?(-|\s)?\d+)(.*)?|(.*)?((S|R)(.)?(R|T)(.)?(\s)?\d+)(.*)?")
    count = 0
    pattern_digitAV = re.compile("^(^[0-9])([A-Z].*)") #If start with one digit and followed by str
    dico = {"HWY" : "HY", "RT " : "I-","INTERSTATE ": "I-","STATE ROUTE ": "I-","STREET": "ST","AVENUE":"AV","AVE":"AV"} 
    for i in range(len(MVA_DF)) : #When finished, len(DF)
        #print("ONE  : ",(MVA_DF["PRIMARY_RD"])[i]," Intersection : ",(MVA_DF["SECONDARY_RD"])[i])
        (MVA_DF["PRIMARY_RD"])[i] = multiple_replace(dico,(MVA_DF["PRIMARY_RD"])[i]) #Get the street name
        (MVA_DF["SECONDARY_RD"])[i] = multiple_replace(dico,(MVA_DF["SECONDARY_RD"])[i]) #Get intersection 
        digit_AV_street = pattern_digitAV.match((MVA_DF["PRIMARY_RD"])[i]) # Check if match with pattern
        digit_AV_Inter =pattern_digitAV.match((MVA_DF["SECONDARY_RD"])[i]) # Check if match with pattern
        if digit_AV_Inter is not None : 
            (MVA_DF["SECONDARY_RD"])[i]= "0" + digit_AV_Inter.group()    #Add a 0 in front if match
        if digit_AV_street is not (None):
            (MVA_DF["PRIMARY_RD"])[i] = "0" + digit_AV_street.group() #Add a 0 in front if match
        if reg.match((MVA_DF["PRIMARY_RD"])[i]):
            #print((MVA_DF["PRIMARY_RD"])[i])
            MVA_DF.drop(i, inplace=True)
            count = count+1
    MVA_DF = MVA_DF.loc[MVA_DF.PCF_VIOL_CATEGORY != '1'] #If driving under drugs or alcohol
    print("count",count)
    print("lenDF: ",len(MVA_DF))
    #MVA_DF = MVA_DF.loc[~(MVA_DF.PRIMARY_RD.str.startswith(r'I-'))] #Take off the accidents owhen on the main Interstates road but not if at intersection
    #TEST = Series(map(lambda x, y: x.startswith(y), MVA_DF.PRIMARY_RD),MVA_DF)
    #MVA_DF = MVA_DF.loc[~(MVA_DF.PRIMARY_RD.str.startswith('SR '))] #Take off the accidents on State Routes (not in OCI df)
    #MVA_DF3 = MVA_DF.loc[~(MVA_DF.PRIMARY_RD.str.startswith('ST-'))] #Take off the accidents on State Routes (not in OCI df)
    #MVA_DF4 = MVA_DF.loc[~(MVA_DF.PRIMARY_RD.str.startswith('ST '))]
    return (MVA_DF)

In [3]:
#Function to aply multiple conditions/regexp on string
def multiple_replace(dico, name_street):
  # Create a regular expression  from the dictionary keys
  regex = re.compile("(%s)" % "|".join(map(re.escape, dico.keys())))

  # For each match, look-up corresponding value in dictionary
  return (regex.sub(lambda mo: dico[mo.string[mo.start():mo.end()]], name_street) )




In [4]:
col_name = ["COLLISION_DATE","COLLISION_TIME","PRIMARY_RD","SECONDARY_RD","CHP_BEAT_TYPE","WEATHER_1","COLLISION_SEVERITY","PRIMARY_COLL_FACTOR","PCF_VIOL_CATEGORY","LIGHTING","ROAD_SURFACE","ROAD_COND_1"]
MVA_DF = pd.DataFrame(data=None, index=None, columns=col_name, dtype=None) #New empty dataframe
size=0
for i in range (4):
    
    year = 2010 + (i+1) #File name
    if year != 2012 : 
        DF = pd.read_csv('CollisionRecords%i.txt' % year,sep = ',', dtype='unicode') #Read the csv file
        DF = DF[col_name]
        DF = cleaning(DF)
        #print(len(DF))
        size += len(DF) #Check size matches 
        MVA_DF = MVA_DF.append(DF)#Add DF to our initialized dataframe

     
#Just to check
print(size)#Just to check


original size :  7840
count 3808
lenDF:  4032
original size :  8858
count 4920
lenDF:  3938
original size :  9503
count 5102
lenDF:  4401
12371


In [5]:
year = 2011 #Only OCI file that I have 
DF_OCI = pd.read_csv('OCI_SD_%i.csv' % year,sep = ',', dtype='unicode') #Read the csv file
DF_OCI= DF_OCI[['oci','street','street_from','street_to','oci_desc']] #Keep only relevant variables
DF_OCI

,oci,street,street_from,street_to,oci_desc
0,49.23,TEBO CT,PICADOR BL,ROBB ROY PL,Fair
1,35.32,04TH AV,K ST,J ST,Poor
2,78.48,MELBOURNE DR,SHAWN AV,IRVINGTON AV,Good
3,34.29,BAYARD ST,REED AV,THOMAS AV,Poor
4,28.28,QUINCE DR,06TH AV,QUINCE ST,Poor
5,75.72,PRINCESS ST,SPINDRIFT DR,CITY BOUNDARY,Good
6,83.43,CLAVELITA ST,ARLISS CT,DEL SUR BL,Good
7,100,IRIS AV,ALBATA CT,ROCHDALE LN,Good
8,51.55,ARLISS CT,CLAVELITA ST,END,Fair
9,45.26,K ST,19TH ST,20TH ST,Fair


In [6]:
def inter(TEMP,intersection_len,intersection,i) : 
    TEMP2 = pd.DataFrame(data=None, index=None, columns= None, dtype=None) #New empty dataframe    
    TEMP2 = TEMP.loc[(TEMP['street_from'] == intersection)|(TEMP['street_to']==(intersection))]  
    if TEMP2.empty : 
        Close_Match_Intersection = (difflib.get_close_matches(intersection, (TEMP['street_from']),n=1))
        Close_Match_To = (difflib.get_close_matches(intersection, (TEMP['street_to']),n=1))
        #print("Close_match : ")
        #print(Close_Match_Intersection,Close_Match_To)
        if (Close_Match_Intersection != []) & (Close_Match_To != []) : 
            TEMP2 = TEMP.loc[(TEMP['street_from'] == Close_Match_Intersection[0])|(TEMP['street_to']== Close_Match_To[0])]
        elif (Close_Match_Intersection != []) : 
            TEMP2 = TEMP.loc[(TEMP['street_from'] == Close_Match_Intersection[0])]
        elif (Close_Match_To != []) : 
            TEMP2 = TEMP.loc[(TEMP['street_to'] == Close_Match_To[0])]
        else : 
            print("Street : ",street_name," Intersection : ",intersection,"INDEX : ",i)
            intersection_len = intersection_len +1
    if TEMP2 is not None : 
        return(TEMP2,intersection_len)

        

In [ ]:
def dataframe(TEMP,TEMP1,DF):
    TEST = pd.DataFrame(data=None, index=None, columns= None, dtype=None)
    TEMP = TEMP.reset_index(drop=True)#From OCI DF
    TEMP1 = MVA_DF.loc[MVA_DF.index == i]
    TEMP1 = TEMP1.reset_index(drop=True)
    a = True
    length = len(TEMP)
    if length ==1 :
        TEST = TEMP.join(TEMP1)
        DF = DF.append(TEST) 
    elif length > 1 : 
        for j in range(len(TEMP)-1) :
            if (TEMP['oci_desc'])[j] != (TEMP['oci_desc'])[j+1]:
                a = False
        if a == True : 
            TEMP = TEMP.loc[[0]]
            TEST = TEMP.join(TEMP1)
            DF = DF.append(TEST)
    if DF is not None :
        return(DF)

In [ ]:
#To be changed - Instead of looping wanted to merge on street name but don't think its possible - So we should make multiple conditions for determining if street + intersection corresponds to the right OCI
#If we have info on To or From  - look directly into the right street-to or street_from - 
#Otherwise, if double answers, check if same OCI_desc - if same just add it - Otherwise should we include it or just exclude these cases ? 

DF = pd.DataFrame(data=None, index=None, columns= None, dtype=None) #Will be final merged DF
street_len = 0 #Var for number of Primary_RD not matching street name
intersection_len = 0 #Var for number of Secondary_RD not matching street_from or street_to
MVA_DF= MVA_DF.reset_index(drop=True)

for i in range(400) : #Loop through all the DF with the Accidents/Collisison
    street_name = (MVA_DF["PRIMARY_RD"])[i] #Get street_name from Primary_RD
    intersection = (MVA_DF["SECONDARY_RD"])[i] #Get the Secondary_RD/Intersection 
    TEMP = DF_OCI.loc[(DF_OCI['street'].str.contains(street_name))] #Get all the street in the OCI that contains or is equal to street_name (from MVA_DF)    
    if TEMP.empty: #If nothing matchs
        street_len = street_len +1
        Close_Match_Street = (difflib.get_close_matches(street_name, (DF_OCI['street']),n=1)) #Function that finds the closest match - Chose to only take one out of the different matches
        if (Close_Match_Street != []) : #If there is a match
            TEMPA = DF_OCI.loc[DF_OCI['street'].str.contains(Close_Match_Street[0])] #Get all the street names from DF_OCI that contain or is equal to this new close match
            TEMP3,intersection_len= inter(TEMPA,intersection_len,intersection,i)
            if not TEMP3.empty : 
                DF = dataframe(TEMP3,i,DF)
        else :
            print("No street match")
            print("Street : ",street_name," Intersection : ",intersection)        
    else : 
        TEMP2,intersection_len= inter(TEMP,intersection_len,intersection,i) 
        DF= dataframe(TEMP2,i,DF)

    print(i)

0
1
Street :  54TH ST  Intersection :  54TH ST 3000 INDEX :  2
2
3
4
Street :  CAMINO SANTA FE  Intersection :  MIRAMAR RD INDEX :  5
5
6
Street :  CAMINO DEL RIO W  Intersection :  KURTZ ST INDEX :  7
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
Street :  FRIARS RD  Intersection :  I-15 INDEX :  27
27
28
Street :  PALM AV  Intersection :  PICCARD AV INDEX :  29
29
Street :  SKYLINE DR  Intersection :  GRIBBLE ST INDEX :  30
30
Street :  PACIFIC HY  Intersection :  TAYLOR ST INDEX :  31
31
32
33
34
Street :  WOODMAN  Intersection :  WOODMAN 200 INDEX :  35
35
36
37
38
39
Street :  ROSECRANS ST  Intersection :  SPORTS ARENA BL INDEX :  40
40
41
42
43
44
45
46
Street :  AVNIDA SUAVIDAD  Intersection :  AVNIDA SUAVIDAD 16265 INDEX :  47
47
48
49
50
51
52
Street :  PERSHING DR  Intersection :  B ST INDEX :  53
53
Street :  BLACK MOUNTAIN RD  Intersection :  BLACK MOUNTAIN RD 10600 INDEX :  54
54
Street :  CAMINO DEL NORTE  Intersection :  I-15 INDEX :  55
55
56
Street :  16TH S

In [ ]:
DF= DF.reset_index(drop=True)
print(street_len,intersection_len,len(DF))
plt.figure()
DF.oci_desc.value_counts().plot(kind='bar') #far from good model but just to have an idea
plt.show()
DF.to_csv('Final_DF.csv')
DF